# Generate pm4py log using ASP Generator

ASP log generator uses the decl model which converts the model into abudction logic programming and pass to the clingo.
Clingo generates the output which is turned into the pm4py log or can be created a .xes file.


In [1]:
!python --version

Python 3.9.12


In [2]:
# importing paths

import sys
import os
import pathlib
import logging

# If you are running from root path, you need specify the module path properly
# or not. Good luck with setting this path if you start your notebook not from the root of this project
SCRIPT_DIR = os.path.join(pathlib.Path("../../src").resolve())
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.declare4py.pm_tasks.log_generation.asp.asp_generator import AspGenerator
from src.declare4py.process_models.decl_model import DeclModel


In [3]:
# Setting up required files

# possible files to test
decl_model_1 = "./decl_files/Response.decl"
decl_model_2 = "./decl_files/MikeModel.decl"
decl_model_3 = "./decl_files/reference10.decl"
decl_model_4 = "./decl_files/diagonisis.decl"

decl_file = os.path.join(pathlib.Path(decl_model_4).resolve())
output_file = os.path.join(pathlib.Path("./generated_exporter.xes").resolve())


In [4]:
# Create Declare model by reading declare model from a file.

model: DeclModel = DeclModel().parse_from_file(decl_file)


In [5]:
#general Setting

# Number of traces that should be generated
num_of_traces = 4

# Minimum and maximum number of events a trace can contain
(num_min_events, num_max_events) = (2, 4)


In [6]:
# Initializing ASP generator with default distributor which is uniform.

logging.basicConfig(level=logging.DEBUG)
asp = AspGenerator(
    model,
    num_of_traces,
    num_min_events,
    num_max_events,
    encode_decl_model=True  #
)


DEBUG:ASP generator:Distribution for traces uniform
DEBUG:ASP generator:traces: 4, events can have a trace min(2) max(4)
INFO:LOG_GENERATOR:Computing distribution
INFO:LOG_GENERATOR:Distribution result Counter({3: 3, 4: 1})


In [7]:
# Violates some constraints for declare model

# NOTE: this examples or cell is only configured for the `decl_model_4` file

# You can add constraint templates as a string array to be violated from the declared model lines.
# You can use the flag `violate_all_constraints_in_subset` in order to tell clingo, to violate all
# templates or some of them( will be decided by clingo).  violate_all_constraints_in_subset= True means
# all the contstraint should be violate which are available in the subset (which you add in the array string/list).

asp.set_negative_traces_len(2)
asp.violate_all_constraints_in_subset = True
asp.add_constraints_subset_to_violate([
    # "Existence[act2] | |",
    # "Existence[act4] | |"
    "Response[Driving_Test, Resit] |A.Grade<=2 | |"
    # "Chain Response[Admission IC, Admission NC] |A.org:group is J |T.org:group is J |61534,61534,s",
    # "Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m",
])


In [8]:
# Generate the traces and parse the result produced by clingo
asp.run()  # Run accets 1 optional value whether to create file for the ASP generated from given declare model


INFO:LOG_GENERATOR:Computing distribution
INFO:LOG_GENERATOR:Distribution result Counter({2: 1, 4: 1})
INFO:LOG_GENERATOR:Computing distribution
INFO:LOG_GENERATOR:Distribution result Counter({4: 2})
DEBUG:ASP generator:Generating negative traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 12
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Start generating traces
DEBUG:ASP generator: Total trace to generate and events: Traces:2, Events: 4, RandFrequency: 0.9
DEBUG:ASP generator: Generating trace:1/2 with events:4, seed:51715140
<block>:33:4-12: info: atom does not occur in any rule head:
  unsat(I)

DEBUG:ASP generator: Traces generated :[assigned_value(b3JnOmdyb3VwENCODEDSTRINGENCODEDSTRING,lowerlowerIlowerlowerEIEEEQUALSIGNNNENCODEDSTRINGENCODEDSTRING,1), assigned_value(lowerlowerTlowerlowerGFjdGljQWNpZAEEEQUALSIGNNNEEEQUALSIGNNNENCODEDSTRINGENCODEDSTRING,66,1), assigned_value(b3JnOmdyb3VwENCODEDST

DEBUG:ASP generator: Clingo Result :SAT
DEBUG:ASP generator:Traces generated. Positive: 2 Neg: 2. Parsing Trace results.
DEBUG:ASP generator:Trace results parsed
DEBUG:ASP generator:Generating Pm4py log
DEBUG:ASP generator:Pm4py generated but not saved yet


In [9]:
# Save file to xes
asp.to_xes(output_file)

/home/mani/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4357.72it/s]
